<a href="https://colab.research.google.com/github/aliakbarbadri/persian-poetry-creator/blob/master/char-gru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time

# Load the data

In [0]:
url = "https://raw.githubusercontent.com/aliakbarbadri/persian-poetry-creator/master/shahname2.txt"
filepath = keras.utils.get_file("shahname.txt", url) 
corpus = open(filepath, 'rb').read().decode(encoding='utf-8')
corpus = corpus.replace("\t"," \t ").replace("\n", " \n ")

In [6]:
vocab = sorted(set(corpus))
print(len(vocab), "chars")

48 chars


In [7]:
vocab

['\t',
 '\n',
 ' ',
 '(',
 ')',
 '«',
 '»',
 '،',
 '؟',
 'ء',
 'آ',
 'أ',
 'ؤ',
 'ئ',
 'ا',
 'ب',
 'ت',
 'ث',
 'ج',
 'ح',
 'خ',
 'د',
 'ذ',
 'ر',
 'ز',
 'س',
 'ش',
 'ص',
 'ض',
 'ط',
 'ظ',
 'ع',
 'غ',
 'ف',
 'ق',
 'ل',
 'م',
 'ن',
 'ه',
 'و',
 'ٔ',
 'پ',
 'چ',
 'ژ',
 'ک',
 'گ',
 'ی',
 '\u200c']

In [0]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in corpus])

In [0]:
seq_length = 1000
examples_per_epoch = len(corpus)//(seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [11]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 1000), (64, 1000)), types: (tf.int64, tf.int64)>

# Model

In [0]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [21]:
model = keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[BATCH_SIZE, None]))
model.add(tf.keras.layers.GRU(rnn_units,return_sequences=True,stateful=True,
                              recurrent_initializer='glorot_uniform',
                              recurrent_regularizer=keras.regularizers.l2(0.01)
                              ))
model.add(tf.keras.layers.GRU(rnn_units,return_sequences=True,stateful=True,
                              recurrent_initializer='glorot_uniform',
                              recurrent_regularizer=keras.regularizers.l2(0.01)
                              ))
model.add(tf.keras.layers.Dense(vocab_size))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           12288     
_________________________________________________________________
gru_4 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
gru_5 (GRU)                  (64, None, 1024)          6297600   
_________________________________________________________________
dense_2 (Dense)              (64, None, 48)            49200     
Total params: 10,297,392
Trainable params: 10,297,392
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [23]:
history = model.fit(dataset, epochs=50)

Epoch 1/20
42/42 [==============================] - 110s 3s/step - loss: 11.1309 - accuracy: 0.2388
Epoch 2/20
42/42 [==============================] - 110s 3s/step - loss: 2.4533 - accuracy: 0.3276
Epoch 3/20
42/42 [==============================] - 110s 3s/step - loss: 2.2267 - accuracy: 0.3535
Epoch 4/20
42/42 [==============================] - 110s 3s/step - loss: 2.1526 - accuracy: 0.3691
Epoch 5/20
42/42 [==============================] - 110s 3s/step - loss: 2.1067 - accuracy: 0.3902
Epoch 6/20
42/42 [==============================] - 110s 3s/step - loss: 2.0316 - accuracy: 0.4117
Epoch 7/20
42/42 [==============================] - 111s 3s/step - loss: 1.9719 - accuracy: 0.4328
Epoch 8/20
42/42 [==============================] - 111s 3s/step - loss: 1.9177 - accuracy: 0.4490
Epoch 9/20
42/42 [==============================] - 110s 3s/step - loss: 1.8599 - accuracy: 0.4652
Epoch 10/20
42/42 [==============================] - 110s 3s/step - loss: 1.8177 - accuracy: 0.4777
Epoch 11

In [24]:
main_model = keras.Sequential()
main_model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[1, None]))
main_model.add(tf.keras.layers.GRU(rnn_units,return_sequences=True,stateful=True,
                              recurrent_initializer='glorot_uniform',
                              recurrent_regularizer=keras.regularizers.l2(0.01)
                              ))
main_model.add(tf.keras.layers.GRU(rnn_units,return_sequences=True,stateful=True,
                              recurrent_initializer='glorot_uniform',
                              recurrent_regularizer=keras.regularizers.l2(0.01)
                              ))
main_model.add(tf.keras.layers.Dense(vocab_size))

main_model.set_weights(model.get_weights())
model = main_model
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            12288     
_________________________________________________________________
gru_6 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
gru_7 (GRU)                  (1, None, 1024)           6297600   
_________________________________________________________________
dense_3 (Dense)              (1, None, 48)             49200     
Total params: 10,297,392
Trainable params: 10,297,392
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string, temperature = 1.0):
  num_generate = 1000
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(idx2char[predicted_id])
  return (start_string + ''.join(text_generated))

In [27]:
print(generate_text(model, start_string=u"که ایران چو باغی ست خرم بهار", temperature=1))

که ایران چو باغی ست خرم بهار 
 نگه کن پرستان بدیگست زان 	 نه خوبت آنچه‌تر دو غفتر نهند 
 برآوردمت زور پاسخ برآب 	 جیو دو چان گر گریره فرخان 
 زنده و نامر روی جست شاه 	 که با ماهن امروز پویست 
 هزاران چنان کرده‌ای مهترای 	 سزادمان آمدم رزمگاه آمد 
 جهان شکسته برینیان و کشت 	 به چندیشه گیتی همی راند اندرزن 
 سخن دو گرین کران ایش پیش 	 کزان تا بر خواستند 
 کهٔ خاک و آید کلاه 	 بباشد بدان تا همی 
 بزد دو با شهریاب فرستاد 	 ربا هم به ماه مهر داز 
 کن لشکر برکشید آورید 
 نشانم نهی گفت آوری خوان 
 به رسم شهریار از جوی نامدار 
 برفتند تا بدخشاه و دارد سپاه 	 گران بی‌آرام اوی زریک 
 بدو گاه بر گورزه خشمع بدگوهش 	 بکوش آذر جهان میده به دل 
 چو از ابلبر بر سپاه پیل 	 بسان زن هنر خواب گید 
 بدان‌ترگ بهر از هستگان 	 چو خوردان برآمد ز کاووس 
 سرش درد وان پس به درید من 	 که بگشادش روشنید و سوار ت
 دوان بد سر سیم بی‌سری 	 فرستگزیده گرد آمد انش 
 ابا پیش آتشاهی بر سپند 	 همان گونه‌کران تخم پیش او بهم 
 ز گر افراسی و زاره بد کند 	 کمر مرزن هرز آورد و با تاج 
 اگر ترکان مرزدار جوی انجمن 
 زمانپی رود شیرا